## Fires Exploration with NASA FIRMS

Notebook for exploring whether fire detections from NASA FIRMS data can be useful for detecting TPA landfill sites.

In [57]:
import folium
import geopandas as gpd
import glob
import os

### Load in data with GeoPandas

Note that you will need to pull down the fires data from Indonesia to get this to run.
GDrive link: https://drive.google.com/file/d/16kGZy1JMky2mO_rtjqLOFevBXZay8uYk/view?usp=sharing

In [2]:
data_dir = '../data'

In [3]:
fires_file = os.path.join(data_dir, 'indonesia_fires.geojson') # this exists in GDrive
tpa_file = os.path.join(data_dir, 'tpa_points.json')
bali_file = os.path.join(data_dir, 'bali.json')

In [4]:
tpa = gpd.read_file(tpa_file)
tpa.head(5)

,id,From Kab,From_#KEC,ID,Kab,Kecamatan,Name,Surface_Ha,TOT_Kg/Day,geometry
0,00000000000000000004,NaN,NaN,5,Klungkung,Nusa Penida,TPA Jungut Batu,1.20,NaN,POINT (115.45941 -8.67096)
1,00000000000000000005,1.0,1.0,6,Klungkung,Nusa Penida,TPA Biaung,1.85,9433.0,POINT (115.49802 -8.67993)
2,00000000000000000003,1.0,3.0,4,Klungkung,Dawan,TPA Sente,1.00,43219.0,POINT (115.45446 -8.53037)
3,00000000000000000006,1.0,4.0,7,Bangli,Bangli,TPA Regional Bangli,0.99,47350.0,POINT (115.36793 -8.35354)
4,00000000000000000008,1.0,5.0,9,Jembrana,Negara,TPA Peh,2.00,38130.0,POINT (114.58329 -8.32794)


In [5]:
print('There are {} TPA sites'.format(len(tpa)))

There are 10 TPA sites


In [6]:
fires = gpd.read_file(fires_file)
fires.head(5)

,acq_date,acq_time,bright_ti5,satellite,instrument,confidence,type,track,bright_ti4,frp,scan,daynight,version,bright_t31,brightness,geometry
0,2020-01-01,0507,294.93,N,VIIRS,n,3,0.39,333.79,4.97,0.44,D,1,NaN,NaN,POINT (127.62113 -3.36016)
1,2020-01-01,0507,294.43,N,VIIRS,n,0,0.39,345.02,34.10,0.46,D,1,NaN,NaN,POINT (127.43561 -1.45792)
2,2020-01-01,0507,295.29,N,VIIRS,n,0,0.37,330.88,3.76,0.40,D,1,NaN,NaN,POINT (126.30626 -3.10995)
3,2020-01-01,0507,290.73,N,VIIRS,n,0,0.40,333.66,3.28,0.47,D,1,NaN,NaN,POINT (128.25844 -3.65959)
4,2020-01-01,0508,300.47,N,VIIRS,n,1,0.41,345.79,14.63,0.50,D,1,NaN,NaN,POINT (127.63496 1.49591)


In [7]:
print('There are {} fires from Jan 1 2019 over Indonesia'.format(len(fires)))

There are 725102 fires from Jan 1 2019 over Indonesia


In [11]:
# clip fires to bali
bali = gpd.read_file(bali_file).set_crs(epsg=4326)
fires_bali = gpd.clip(fires, bali)

In [14]:
fires_bali.head(5)

,acq_date,acq_time,bright_ti5,satellite,instrument,confidence,type,track,bright_ti4,frp,scan,daynight,version,bright_t31,brightness,geometry
1593,2020-01-18,1732,287.34,N,VIIRS,n,3,0.37,302.63,0.57,0.40,N,1,NaN,NaN,POINT (115.50683 -8.34106)
1764,2020-01-19,1713,287.66,N,VIIRS,n,3,0.47,300.43,0.89,0.45,N,1,NaN,NaN,POINT (115.50710 -8.34080)
2227,2020-01-22,1757,287.82,N,VIIRS,n,3,0.44,298.12,1.02,0.39,N,1,NaN,NaN,POINT (115.50869 -8.34318)
2427,2020-01-23,1738,288.24,N,VIIRS,n,3,0.36,305.38,0.75,0.38,N,1,NaN,NaN,POINT (115.50848 -8.34191)
3082,2020-01-27,1804,285.93,N,VIIRS,n,3,0.49,297.10,0.75,0.50,N,1,NaN,NaN,POINT (115.50705 -8.34189)


In [15]:
print('Clipped to {} fires in Bali'.format(len(fires_bali)))

Clipped to 767 fires in Bali


### Do fires fall within TPA sites?

In [16]:
# buffer TPA points into polygons
buffer_width = 0.01 # degrees
tpa['geometry'] = tpa.set_crs(epsg=4326).buffer(buffer_width).envelope

<ipython-input-16-4b367d6d651b>:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tpa['geometry'] = tpa.set_crs(epsg=4326).buffer(buffer_width).envelope


In [17]:
# join fires that fall within the bounding boxes of the TPA sites
fires_tpa = gpd.sjoin(fires_bali, tpa, how='inner', op='intersects')

In [18]:
# only keep fire detections with high confidence (marked nominal "n")
fires_tpa = fires_tpa[fires_tpa['confidence'] == 'n']

In [19]:
fires_tpa.head(5)

,acq_date,acq_time,bright_ti5,satellite,instrument,confidence,type,track,bright_ti4,frp,...,index_right,id,From Kab,From_#KEC,ID,Kab,Kecamatan,Name,Surface_Ha,TOT_Kg/Day
52748,2020-06-06,1754,NaN,1,VIIRS,n,None,0.43,NaN,2.30,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0
71010,2020-08-21,0624,NaN,1,VIIRS,n,None,0.61,NaN,3.20,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0
474903,2019-10-19,1739,290.06,N,VIIRS,n,0,0.36,338.55,5.59,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0
474904,2019-10-19,1739,287.11,N,VIIRS,n,0,0.36,338.31,5.99,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0
542913,2019-11-22,1701,292.38,N,VIIRS,n,0,0.59,305.33,0.55,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0


In [20]:
print('Found {} fires that fall within TPA sites'.format(len(fires_tpa)))

Found 7 fires that fall within TPA sites


### Visualize fires on a Folium map

In [87]:
# create a folium map centered on Bali
# use Google satellite hybrid view as our basemap
m = folium.Map(location=(-8.4199969,114.9164863), 
               tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', 
               attr='Google',
               name='Google Satellite Hybrid')

# add TPA bounding boxes
tpa_locs = folium.GeoJson(tpa, name='TPA Locations').add_to(m)

# add Bali fires
def add_circle(f):
    lon, lat = f['geometry'].x, f['geometry'].y
    circle = folium.CircleMarker(location=(lat, lon), 
                                 radius=2, 
                                 weight=f['frp']/5, 
                                 color='red',
                                 fillColor='red',
                                 opacity=0.8,
                                 fillOpacity=0.8).add_to(m)
    
for ridx, row in fires_bali.iterrows():
    add_circle(row)

In [88]:
display(m)

In [90]:
# save map as HTML
m.save('../maps/FIRMS_fires_bali.html')